In [6]:
!pip install groq

In [ ]:
import os

os.environ["GROQ_API_KEY"] = ""

In [8]:
from groq import Groq

client = Groq()


# Classe Agent
Igual ao código original

In [9]:
class Agent:
  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = []
    if self.system is not None:
      self.messages.append({"role": "system", "content": self.system})

  def __call__(self, message=""):
    if message:
      self.messages.append({"role": "user", "content": message})
    result = self.execute()
    self.messages.append({"role": "assistant", "content": result})
    return result

  def execute(self):
    completion = self.client.chat.completions.create(
        messages=self.messages,
        model="llama3-8b-8192",
    )
    return completion.choices[0].message.content


# Prompt do sistema
Para a minha atividade prática, eu enfatizei que o agente não deve fingir que sabe a resposta de alguma ação, e que ele deverá apenas fazer uma chamada para ação por vez.

Além disso, modifiquei os exemplos para serem compatíveis com o novo contexto de gerar respostas sobre remédios para doenças.

In [34]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Never assume the answer of an observation, alway call for the action.
Always call for one action at a time.
Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_medication:
e.g. get_medication: Diabetes
returns the name, unit and amount of the medication per pacient kg.

Example session:

Question: What is the prescription for diabetes on a male with a body weight of 80kg?
Thought: I need to find the medication for diabetes, and it's data.
PAUSE

Action: get_medication: Diabetes
PAUSE

You will be called again with this:

Observation: Medication: Insulin, Unit: IU, Amount: 0.4/kg/day

Thought: I need to multiply 0.4 by 80 to get the amount of insulin per kg per day.
PAUSE

Action: calculate: 0.4 * 80
PAUSE

You will be called again with this:

Observation: 32

If you have the answer, output it as the Answer.

Answer: A pacient with a body weight of 80kg should take something arount 32IU (International Units) of insulin per day.

Now it's your turn:
""".strip()

# Ferramentas
Precisei fazer uma pesquisa sobre algumas doenças específicas, que tivessem tratamentos comuns e baseados em medicações.

Organizei a função de forma que o agente possa receber informações gerais para calcular a quantidade de medicação que deverá tomar/aplicar.


In [26]:
# tools
def calculate(operation: str) -> float:
  return eval(operation)

def get_medication(illness: str) -> str:
    match illness.lower():
        case "diabetes":
            return "Medication: Insulin, Unit: IU, Amount: 0.4/kg/day"
        case "hypertension":
            return "Medication: Lisinopril, Unit: mg, Amount: 10-20 mg/day"
        case "asthma":
            return "Medication: Albuterol, Unit: puffs, Amount: 2 puffs as needed"
        case "hyperlipidemia":
            return "Medication: Atorvastatin, Unit: mg, Amount: 10-80 mg/day"
        case "depression":
            return "Medication: Fluoxetine, Unit: mg, Amount: 20 mg/day"
        case "anemia":
            return "Medication: Iron, Unit: mg, Amount: 3 mg/kg/day"
        case "infection":
            return "Medication: Amoxicillin, Unit: mg, Amount: 20-40 mg/kg/day"
        case "pain":
            return "Medication: Ibuprofen, Unit: mg, Amount: 10 mg/kg/dose every 6-8 hours"
        case _:
            return "This illness is not in our database"


In [31]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "get_medication"]
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)
        print("-"*50)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            try:
              chosen_tool = action[0][0]
              arg = action[0][1]
            except IndexError:
              continue

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            print("-"*50)
            continue

        if "Answer" in result:
            break


# Teste básico

Nesse exemplo foi perguntado qual seria a dose ideal de medicação para um paciente com anemia, que pesa 87 quilos. Após isso,
o agente detectou a necessidade de chamar a medicação para a anemia, e suas informações, a execução da função gerou a seguinte
observação: Medicamento: Ferro, Unidade: mg, Quantidade: 3 mg/kg/dia. Esse dado está claramente não estruturado, mas o
agente entendeu que deveria executar a ação de multiplicação 87 × 3 e assim o fez, recebendo uma observação de 261, o que é
um dado inteiro, e estruturado, mas não claramente contextualizado, entretanto, o agente finalizou sua resposta, concluindo que o
paciente deveria tomar 261mg de ferro por dia.

In [28]:
agent_loop(
    max_iterations=10,
    system=system_prompt,
    query="What amount of medication should a pacient with 87kg take for his anemia?"
    )

Thought: I need to find the medication for anemia, and it's data.
PAUSE

Action: get_medication: Anemia
PAUSE

Please wait for the observation.
--------------------------------------------------
Observation: Medication: Iron, Unit: mg, Amount: 3 mg/kg/day
--------------------------------------------------
Thought: I need to multiply 3 by 87 to get the amount of iron per day.
PAUSE

Action: calculate: 3 * 87
PAUSE
--------------------------------------------------
Observation: 261
--------------------------------------------------
Thought: A patient with a body weight of 87kg should take 261mg of iron per day.
Answer: A patient with a body weight of 87kg should take 261mg of iron per day.
--------------------------------------------------


# Teste complexo

Aqui foi perguntado sobre 3 doenças de uma vez, e o agente foi capaz de separar a pergunta em partes, resolver cada
parte individualmente e retornar uma resposta completa.

O único problema foi o cálculo para o lisonopril, onde o agente apenas somou as duas respostas para o remédioa anterior.

In [38]:
## Challenge
agent_loop(max_iterations=15, system=system_prompt, query="What amount of medication should a pacient with 87kg take? He has diabetes, anemia and hypertension!")


Thought: I need to get the medication for diabetes, anemia, and hypertension, and it's data.
PAUSE
--------------------------------------------------
Action: get_medication: Diabetes
PAUSE
--------------------------------------------------
Observation: Medication: Insulin, Unit: IU, Amount: 0.4/kg/day
--------------------------------------------------
Thought: I need to get the medication for anemia and hypertension, and their data.
PAUSE
--------------------------------------------------
Action: get_medication: Anemia
PAUSE
--------------------------------------------------
Observation: Medication: Iron, Unit: mg, Amount: 3 mg/kg/day
--------------------------------------------------
Thought: I now have the medication for diabetes and anemia, and their data. I should calculate the total amount of insulin and iron the patient should take.
PAUSE
--------------------------------------------------
Action: get_medication: Hypertension
PAUSE
-------------------------------------------------